<a href="https://colab.research.google.com/github/ArunVignesh75/Machine-Learning/blob/main/RoboFlow_with_Yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler

In [ ]:
from roboflow import Roboflow
from PIL import Image
import os

In [ ]:
# Replace "api_key", "assemblyannotations", and "1" with your actual values
api_key = "Api_Key"
project_name = "assemblyannotations"
version = "1"

# Specify the path to the folder containing images
image_folder_path = r"/content/drive/MyDrive/Arun_25_01_24"

# Specify the dimensions for resizing to 1080p
new_width = 1920
new_height = 1080

rf = Roboflow(api_key=api_key)
project = rf.workspace().project(project_name)
model = project.version(version).model

# List all image files in the folder
image_files = [f for f in os.listdir(image_folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

for image_file in image_files:
    # Construct the full path to the image
    image_path = os.path.join(image_folder_path, image_file)

    # Open and resize the image to 1080p
    img = Image.open(image_path)
    resized_img = img.resize((new_width, new_height))
    resized_img.save("resized_image.jpg")

    # Infer on the resized image
    result = model.predict("resized_image.jpg", confidence=40, overlap=30).json()

    # Process the result and save annotations in .txt format
    annotations = []
    for prediction in result['predictions']:
        class_id = prediction['class_id']
        x_center = prediction['x']
        y_center = prediction['y']
        width = prediction['width']
        height = prediction['height']

        # Convert to YOLO format
        x_center_normalized = x_center / new_width
        y_center_normalized = y_center / new_height
        width_normalized = width / new_width
        height_normalized = height / new_height

        annotations.append(f"{class_id} {x_center_normalized} {y_center_normalized} {width_normalized} {height_normalized}")

    # Save annotations to a .txt file in the same folder as the image
    annotation_file_path = os.path.join(image_folder_path, f"{os.path.splitext(image_file)[0]}.txt")
    with open(annotation_file_path, "w") as annotation_file:
        annotation_file.write("\n".join(annotations))

    print(f"Annotations saved for {image_file} in {annotation_file_path}")
